In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
import xgboost
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
df=pd.read_csv('../input/hackerearth-how-not-to-lose-a-customer-in-10-days/train.csv',na_values=['?','-999','Error','xxxxxxxx'])
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
df=df.drop(['customer_id','Name','security_no','referral_id','last_visit_time'],axis=1)

In [ ]:
df['churn_risk_score'].value_counts()

In [ ]:
df['churn_risk_score'].hist()
plt.show()

In [ ]:
df.info()

In [ ]:
for i in df.columns:
    if df[i].dtype=='float64':
        df[i]=df[i].fillna(df[i].mean())
    else:
        df[i]=df[i].fillna(method='ffill')

In [ ]:
df.isnull().sum()

In [ ]:
df.fillna('Both',inplace=True)

In [ ]:
df['joining_date']=df['joining_date'].apply(lambda x:int(x.split('-')[0]))

In [ ]:
df.rename(columns={'joining_date':'joining_year'},inplace=True)

In [ ]:
df1=pd.get_dummies(df,drop_first=True)

In [ ]:
x=df1.drop('churn_risk_score',axis=1)
y=df1['churn_risk_score']

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,stratify=y)

In [ ]:
xgb = xgboost.XGBClassifier(tree_method= 'gpu_hist')

In [ ]:
xgb.fit(x_train,y_train)

In [ ]:
pred=xgb.predict(x_test)

In [ ]:
print(f1_score(y_test,pred,average='micro'))

In [ ]:
# params={'learning_rate':[0.05,0.1,0.15],
#       'max_depth':[2,4,6,8],
#       'min_child_weight':[1,3,5],
#       'gamma':[0.2,0.3,0.4,0.5],
#       'colsample_bytree':[0.4,0.5,0.7]}

In [ ]:
# model=RandomizedSearchCV(xgb,param_distributions=params,n_iter=100,cv=3,n_jobs=-1)
# model.fit(x,y)

In [ ]:
# model.best_params_

In [ ]:
model1 = xgboost.XGBClassifier(min_child_weight= 3,
 max_depth= 4,
 learning_rate= 0.1,
 gamma= 0.5,
 colsample_bytree= 0.7)

In [ ]:
model1.fit(x,y)

In [ ]:
test=pd.read_csv('../input/hackerearth-how-not-to-lose-a-customer-in-10-days/test.csv',na_values=['?','-999','Error','xxxxxxxx'])
test.head()

In [ ]:
test=test.drop(['customer_id','Name','security_no','referral_id','last_visit_time'],axis=1)

In [ ]:
for i in test.columns:
    if test[i].dtype=='float64':
        test[i]=test[i].fillna(test[i].mean())
    else:
        test[i]=test[i].fillna(method='ffill')

In [ ]:
test['joining_date']=test['joining_date'].apply(lambda x:int(x.split('-')[0]))

In [ ]:
test.rename(columns={'joining_date':'joining_year'},inplace=True)

In [ ]:
test1=pd.get_dummies(test,drop_first=True)

In [ ]:
result = model1.predict(test1)

In [ ]:
test1 =pd.read_csv('../input/hackerearth-how-not-to-lose-a-customer-in-10-days/test.csv',na_values=['?','-999','Error','xxxxxxxx'])

In [ ]:
submission = pd.DataFrame({
    'customer_id': test1['customer_id'],
    'churn_risk_score': result,
})

In [ ]:
print(submission['churn_risk_score'].value_counts())

In [ ]:
submission.to_csv('sub.csv', index=False)